# Kernel PCA

Kernel, es una función matemática que permite transformar los datos de una dimensión a otra más alta. Esto permite la clasificación de datos cuando éstos no son linealmente separables en su espacio original.

**Ejemplo: Datos originales vs Datos transformados con Kernel**

![Aplicación de una función de Kernel](../imgs/kernel_f.png)


**Los kernes más comunes**:
- Lineales $k(x, y) = x*y$ ➡️ Combinaciones lineales entre variables.
- Polinomiales $k(x, y) = (x*y)^p$ ➡️ Polinomios y exponentes, dan mayor flexibilidad en relaciones no lineales.
- Gaussianos (RBF) $k(x, y) = e^{- \frac {||x-y||^2}{2\sigma ^2}}$ ➡️ Crea estructuras complejas.

![Visualización de las diferentes funciones de kernel](../imgs/funciones_kernel.png)


📌 Cuando no podamos partir las clases en separaciones lineales, podemos utilizar otras opciones, como kernels polinomiales o RBF (Radial Basis Function) que nos pueden dar muy buenos resultados.


Scikit-Learn nos proporciona la funcionalidad `KernelPCA` como una extensión de PCA clásico, pero:

- Utiliza un kernel no lineal para proyectar los datos en un nuevo espacio o dimensión.
- Aplica PCA en ese nuevo espacio (donde la varianza puede ser mayor).

**¿Cuándo usar KernelPCA?**

- Cuando los datos no son linealmente separables.
- Como paso previo a un modelo lineal (regresión logística).
- Cuando PCA clásico no mejora el modelo.

Importamos la librerias para el ejemplo.

In [1]:
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt

# Reducción de dimensionalidad
from sklearn.decomposition import KernelPCA

# Modelo de regresión logistica
from sklearn.linear_model import LogisticRegression
# Estandarizador
from sklearn.preprocessing import StandardScaler
# Divir dataset en entrenamiento y pruebas
from sklearn.model_selection import train_test_split

In [2]:
%run 0.0-ml_professional-setup.ipynb

In [3]:
file_path = path.data_raw_dir("heart.csv")
df = pd.read_csv(file_path)

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


Separamos las features y el target en dataframes diferentes:

In [4]:
df_features = df.drop(columns=["target"])
s_target = df["target"]
display(df_features.head(), s_target.head())

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2


0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

Estandarización y división de los datos en conjuntos de entrenamiento y pruebas:

In [5]:
df_features = StandardScaler().fit_transform(df_features)

X_train, X_test, y_train, y_test = train_test_split(df_features, s_target, test_size=.2, random_state=42)
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

X_train: (820, 13) y_train: (820,)
X_test: (205, 13) y_test: (205,)


Entrenamiento de KernelPCA
- `n_components`: Dimensión (features) a la que queremos reducir el dataset.
- `kernel`: Tipo de kernel que va implementar.

In [6]:
kpca = KernelPCA(n_components=4, kernel="linear")
kpca.fit(X_train)

KernelPCA(n_components=4)

Entrenamos el modelo de regresión logistica con los componentes generados con KernelPCA.

In [7]:
lr_kpca = LogisticRegression()

# Reduce y transforma los datos de entrenamiento y pruebas a los 3 componentes generados con PCA:
X_train_trans = kpca.transform(X_train)
X_test_trans = kpca.transform(X_test)

# Entrenamiento el modelo:
lr_kpca.fit(X_train_trans, y_train)

score = lr_kpca.score(X_test_trans, y_test)
print(f"Score con KernelPCA: {score:.2%}")

Score con KernelPCA: 80.49%
